In [7]:
import json
from pathlib import Path

import geopandas as gpd
import fiona
import numpy as np
import pandas as pd
import pycountry

from climada.util.api_client import Client
from climada.entity import LitPop, ImpactFuncSet, ImpactFunc
from climada.hazard import Hazard
from climada.engine import Impact
from climada.entity import Exposures


notebook_dir = %pwd

# Go up one directory
BASE_DIR = Path(notebook_dir).parent

# Define the paths for exposures and hazards
DATA_DIR = BASE_DIR / "data"
EXPOSURES_DIR = BASE_DIR / "data" / "exposures"
HAZARDS_DIR = BASE_DIR / "data" / "hazards"
REQUIREMENTS_DIR = BASE_DIR / "requirements"
MAP_DIR = BASE_DIR / "src" / "components" / "map"
GADM41_EGY_PATH = REQUIREMENTS_DIR / "gadm41_EGY.gpkg"
GADM41_THA_PATH = REQUIREMENTS_DIR / "gadm41_THA.gpkg"
country_name = "Thailand"
hazard_type = "river_flood"
climate_scenario = "historical"


client = Client()

In [2]:
# List DataTypeInfos
data_type_infos= client.list_data_type_infos()
data_type_infos

[DataTypeInfo(data_type='litpop', data_type_group='exposures', status='active', description='A global high-resolution asset exposure dataset produced using “lit population” (LitPop), a globally consistent methodology to disaggregate asset value data proportional to a combination of nightlight intensity and geographical population data. Exposure data for population, asset values and productive capital at 4km spatial resolution globally, consistent across country borders. The dataset offers value for manifold use cases, including globally consistent economic disaster risk assessments and climate change adaptation studies, especially for larger regions, yet at considerably high resolution.', properties=[{'property': 'spatial_coverage', 'mandatory': True, 'description': 'spatial coverage by country (one file each) or global (in one file), for some also other groups, such as tropical cyclone genesis basin'}, {'property': 'climada_version', 'mandatory': True, 'description': 'the version of C

In [6]:
# List DataType Exposures
data_type_exposures = [data_type_info.data_type for data_type_info in data_type_infos if data_type_info.data_type_group == 'exposures']
data_type_exposures

['litpop', 'crop_production', 'crops']

In [10]:
# List DataType Hazards
data_type_hazards = [data_type_info.data_type for data_type_info in data_type_infos if data_type_info.data_type_group == 'hazard']
data_type_hazards

['tropical_cyclone',
 'wildfire',
 'river_flood',
 'storm_europe',
 'relative_cropyield',
 'earthquake',
 'flood',
 'hail']

In [3]:
# Available country Exposures and Hazard data types
dataset_infos = client.list_dataset_infos(
    properties={
        'country_name': country_name,
    }
)

exposure_data_types= list(set([dataset_info.data_type.data_type for dataset_info in dataset_infos if dataset_info.data_type.data_type_group == 'exposures']))
hazard_data_types = list(set([dataset_info.data_type.data_type for dataset_info in dataset_infos if dataset_info.data_type.data_type_group == 'hazard']))

print(f'exposure_data_types: {exposure_data_types}')
print(f'hazard_data_types: {hazard_data_types}')

exposure_data_types: ['litpop']
hazard_data_types: ['wildfire', 'earthquake', 'tropical_cyclone', 'flood', 'river_flood']


In [4]:
# Available country Hazard DatasetInfos
hazard_dataset_infos = client.list_dataset_infos(
    properties={
        'data_type': hazard_type,
        'country_name': country_name,
        'climate_scenario': climate_scenario
    }
)
hazard_dataset_infos

[DatasetInfo(uuid='f3f61cd8-d39b-473c-9acb-fd5bffac52e0', data_type=DataTypeShortInfo(data_type='river_flood', data_type_group='hazard'), name='river_flood_150arcsec_hist_THA_1980_2000', version='v2', status='active', properties={'res_arcsec': '150', 'climate_scenario': 'historical', 'year_range': '1980_2000', 'climada_version': 'v3.1.2', 'spatial_coverage': 'country', 'date_creation': '2022-7-6', 'country_iso3alpha': 'THA', 'country_name': 'Thailand', 'country_iso3num': '764'}, files=[FileInfo(uuid='f3f61cd8-d39b-473c-9acb-fd5bffac52e0', url='https://data.iac.ethz.ch/climada/f3f61cd8-d39b-473c-9acb-fd5bffac52e0/river_flood_150arcsec_hist_THA_1980_2000.hdf5', file_name='river_flood_150arcsec_hist_THA_1980_2000.hdf5', file_format='hdf5', file_size=5679244, check_sum='md5:102d02e170464a1bb15a745edfd8832c')], doi=None, description='Historical CLIMADA flood hazard by country and by 20 year range at a 150 arcsec resolution. The hazard is based on annual spatially explicit flood depth and fl

In [5]:
# Get Exposure and Exposure gdf
# Available Exposures: ['litpop']
exposure = client.get_litpop(
    country=country_name,
    exponents=(1,1),
    dump_dir=EXPOSURES_DIR,
)
exposure_gdf = exposure.gdf
exposure_gdf

,value,geometry,latitude,longitude,region_id,impf_
0,2.268901e+06,POINT (99.06250 7.60417),7.604167,99.062500,764,1
1,2.648491e+06,POINT (99.06250 7.56250),7.562500,99.062500,764,1
2,1.217022e+06,POINT (99.06250 7.52083),7.520833,99.062500,764,1
3,2.643969e+05,POINT (99.10417 7.47917),7.479167,99.104167,764,1
4,1.177234e+06,POINT (99.06250 7.64583),7.645833,99.062500,764,1
...,...,...,...,...,...,...
24966,7.343259e+05,POINT (101.10417 5.68750),5.687500,101.104167,764,1
24967,2.069925e+05,POINT (101.14583 5.68750),5.687500,101.145833,764,1
24968,1.702074e+05,POINT (101.18750 5.68750),5.687500,101.187500,764,1
24969,1.674955e+05,POINT (101.10417 5.64583),5.645833,101.104167,764,1


In [6]:
# Get Hazard
# Available Hazards: ['river_flood', 'tropical_cyclone', 'wildfire', 'flood', 'earthquake']

hazard = client.get_hazard(
    hazard_type=hazard_type,
    properties={
        "country_name": country_name,
        "climate_scenario": climate_scenario,
    },
    dump_dir=HAZARDS_DIR,
)

In [32]:
# Calculate Impact

hazard.intensity_thres = 1
impact_function = ImpactFunc()
impact_function.haz_type = "RF"
impact_function.intensity_unit = "m"
impact_function.id = 3
impact_function.name = "Flood Europe JRC Residential"
impact_function.intensity = np.array(
    [0.0, 0.5, 1.0, 1.5, 2.0, 3.0, 4.0, 5.0, 6.0, 12.0]
)
impact_function.mdd = np.array(
    [0.00, 0.25, 0.40, 0.50, 0.60, 0.75, 0.85, 0.95, 1.00, 1.00]
)
impact_function.mdr = np.array(
    [0.000, 0.250, 0.400, 0.500, 0.600, 0.750, 0.850, 0.950, 1.000, 1.000]
)
impact_function.paa = np.ones(len(impact_function.intensity))
impact_function_set = ImpactFuncSet()
# impact_function_set = ImpactFuncSet([impact_function])
impf_id = 3

impact_function_set.append(impact_function)
exposure.gdf[f"impf_{impact_function.haz_type}"] = impf_id
exposure.impact_funcs#s = impact_function_set
impact = Impact()
impact.calc(exposure, impact_function_set, hazard, save_mat=True)

impact
# impact_function_set.check()

2023-11-21 12:26:06,024 - climada.engine.impact - WARNING - The use of Impact().calc() is deprecated. Use ImpactCalc().impact() instead.


In [10]:
# adm_gdf = gdf = gpd.read_file(GADM41_EGY_PATH, layer=0) # access layers by index (ex: 1) or name (ex: 'ADM_ADM_0')
adm_gdf = gdf = gpd.read_file(GADM41_THA_PATH, layer=2) # access layers by index (ex: 1) or name (ex: 'ADM_ADM_0')
adm_gdf.head()

,GID_2,GID_0,COUNTRY,GID_1,NAME_1,NL_NAME_1,NAME_2,VARNAME_2,NL_NAME_2,TYPE_2,ENGTYPE_2,CC_2,HASC_2,geometry
0,THA.1.1_1,THA,Thailand,THA.1_1,Amnat Charoen,จังหวัดอำนาจเจริญ,Chanuman,NA,อำเภอชานุมาน,Amphoe,District,3702,TH.AC.CH,"MULTIPOLYGON (((105.04093 16.11617, 105.04107 ..."
1,THA.1.2_1,THA,Thailand,THA.1_1,Amnat Charoen,จังหวัดอำนาจเจริญ,Hua Taphan,NA,อำเภอหัวตะพาน,Amphoe,District,3706,TH.AC.HU,"MULTIPOLYGON (((104.58716 15.60595, 104.58696 ..."
2,THA.1.3_1,THA,Thailand,THA.1_1,Amnat Charoen,จังหวัดอำนาจเจริญ,Lu Amnat,NA,อำเภอลืออำนาจ,Amphoe,District,3707,TH.AC.LU,"MULTIPOLYGON (((104.79344 15.69906, 104.79359 ..."
3,THA.1.4_1,THA,Thailand,THA.1_1,Amnat Charoen,จังหวัดอำนาจเจริญ,Muang Amnat Charoen,NA,อำเภอเมืองอำนาจเจริญ,Amphoe,District,3701,TH.AC.AM,"MULTIPOLYGON (((104.78572 15.71766, 104.78545 ..."
4,THA.1.5_1,THA,Thailand,THA.1_1,Amnat Charoen,จังหวัดอำนาจเจริญ,Pathum Ratwongsa,NA,อำเภอปทุมราชวงศา,Amphoe,District,3703,TH.AC.PA,"MULTIPOLYGON (((104.81931 15.78355, 104.81943 ..."


In [11]:
# List all layers in the GeoPackage
layer_names = gpd.io.file.fiona.listlayers(GADM41_THA_PATH)
# layer_names = gpd.io.file.fiona.listlayers(GADM41_EGY_PATH)
for name in layer_names:
    print(name)


ADM_ADM_0
ADM_ADM_1
ADM_ADM_2
ADM_ADM_3


In [6]:
filepath = DATA_DIR / "dev" / "gadm41_EGY.gpkg"
layer = 2
exposure_filepath = EXPOSURES_DIR / "LitPop_150arcsec_EGY.hdf5"
exposure = Exposures().from_hdf5(exposure_filepath)
exp_gdf = exposure.gdf
adm_gdf = gpd.read_file(filepath, layer=layer) # access layers by index (ex: 1) or name (ex: 'ADM_ADM_0')

joined_gdf = gpd.sjoin(exp_gdf, adm_gdf, how="left", predicate='within')
aggregated_values = joined_gdf.groupby(f'GID_{layer}')['value'].sum().reset_index()
adm_gdf = adm_gdf.merge(aggregated_values, on=f'GID_{layer}', how='left')
adm_gdf['value'] = adm_gdf['value'].fillna(0)
adm_gdf

resp_gdf = adm_gdf[[f'GID_{layer}', 'geometry', 'value']]
resp_gdf


,GID_2,geometry,value
0,EGY.1.1_1,"MULTIPOLYGON (((31.30377 30.99935, 31.31002 30...",1.709652e+09
1,EGY.1.2_1,"MULTIPOLYGON (((31.99799 31.22747, 31.98231 31...",1.605807e+08
2,EGY.1.3_1,"MULTIPOLYGON (((31.39126 30.93035, 31.38016 30...",2.439367e+09
3,EGY.1.4_1,"MULTIPOLYGON (((31.39234 31.01250, 31.37525 31...",0.000000e+00
4,EGY.1.5_1,"MULTIPOLYGON (((31.39234 31.01250, 31.38253 31...",1.333697e+09
...,...,...,...
338,EGY.27.14_1,"MULTIPOLYGON (((31.70411 26.56144, 31.70938 26...",0.000000e+00
339,EGY.27.15_1,"MULTIPOLYGON (((31.50795 26.85638, 31.51352 26...",1.267270e+09
340,EGY.27.16_1,"MULTIPOLYGON (((31.49783 26.74574, 31.48510 26...",4.249094e+08
341,EGY.27.17_1,"MULTIPOLYGON (((31.44358 26.94474, 31.48056 26...",1.612533e+09


In [12]:
def get_iso3_country_code(country_name: str) -> str:
    try:
        country = pycountry.countries.search_fuzzy(country_name)[0]
        return country.alpha_3
    except LookupError:
        print(f"No ISO3 code found for '{country_name}'. Please check the country name.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def generate_exposure_geojson(exposure: Exposures, country_name: str):
    try:
        exposure_gdf = exposure.gdf
        country_iso3 = get_iso3_country_code(country_name)

        GADM41_filename = Path(REQUIREMENTS_DIR) / f"gadm41_{country_iso3}.gpkg"
        layers = [0, 1, 2]

        all_layers_geojson = {"type": "FeatureCollection", "features": []}

        for layer in layers:
            try:
                admin_gdf = gpd.read_file(filename=GADM41_filename, layer=layer)
                joined_gdf = gpd.sjoin(exposure_gdf, admin_gdf, how="left", predicate="within")
                aggregated_values = joined_gdf.groupby(f"GID_{layer}")["value"].sum().reset_index()
                admin_gdf = admin_gdf.merge(aggregated_values, on=f"GID_{layer}", how="left")
                admin_gdf["value"] = admin_gdf["value"].fillna(0)
                admin_gdf_filtered = admin_gdf[[f"GID_{layer}", "geometry", "value"]]

                # Convert each layer to a GeoJSON Feature and add it to the collection
                layer_features = admin_gdf_filtered.__geo_interface__["features"]
                for feature in layer_features:
                    feature["properties"]["layer"] = layer
                    all_layers_geojson["features"].append(feature)

            except FileNotFoundError:
                print("debug", f"File not found: {GADM41_filename}")
            except Exception as e:
                print("debug", f"An error occurred while processing layer {layer}: {e}")

        # Save the combined GeoJSON file
        map_data_filepath = MAP_DIR / "exposures_geodata.json"
        with open(map_data_filepath, "w") as f:
            json.dump(all_layers_geojson, f)

    except AttributeError as e:
        print("debug", f"Invalid Exposure object: {e}")
    except Exception as e:
        print("debug", f"An unexpected error occurred: {e}")


generate_exposure_geojson(exposure, country_name)